In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruT5-base")

model = AutoModelForSeq2SeqLM.from_pretrained("sberbank-ai/ruT5-base")

df = pd.read_csv("assessed.csv")

In [4]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class ToxicDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels, decoder_attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
        self.decoder_attention_mask = decoder_attention_mask
        
    def __len__(self):
        return int(self.input_ids.size(dim=0))
    
    def __getitem__(self, i):
        return {"input_ids": self.input_ids[i], "attention_mask": self.attention_mask[i], "labels": self.labels[i], "decoder_attention_mask": self.decoder_attention_mask[i]}
    
train_df, eval_df = train_test_split(df, test_size=0.2)

In [5]:
prefix = tokenizer(train_df["detoxified"].values.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512, pad_to_multiple_of=8)
train_input_ids, train_attention_mask = prefix["input_ids"], prefix["attention_mask"]
suffix = tokenizer(train_df["toxic"].values.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512)
train_labels, train_decoder_attention_mask = suffix["input_ids"], suffix["attention_mask"]

prefix = tokenizer(eval_df["detoxified"].values.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512, pad_to_multiple_of=8)
eval_input_ids, eval_attention_mask = prefix["input_ids"], prefix["attention_mask"]
suffix = tokenizer(eval_df["toxic"].values.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512)
eval_labels, eval_decoder_attention_mask = suffix["input_ids"], suffix["attention_mask"]

In [6]:
train_dataset = ToxicDataset(train_input_ids, train_attention_mask, train_labels, train_decoder_attention_mask)
eval_dataset = ToxicDataset(eval_input_ids, eval_attention_mask, eval_labels, eval_decoder_attention_mask)

In [7]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [8]:
args = Seq2SeqTrainingArguments(
    output_dir = "assessed_model",
    evaluation_strategy = "steps",
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 8,
    num_train_epochs = 3,
    save_strategy = "steps",
    save_steps = 50,
    fp16 = True,
    eval_steps = 25,
    dataloader_num_workers = 4,
    group_by_length = True,
    generation_max_length=256
)

trainer = Seq2SeqTrainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer
)

Using amp half precision backend


In [9]:
trainer.train()

/home/jupyter-black_samorez/.conda/envs/megatron/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5558
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 258


Step,Training Loss,Validation Loss
25,No log,1.770770
50,No log,0.964918
75,No log,0.802849
100,No log,0.746391
125,No log,0.721609
150,No log,0.698796
175,No log,0.683865
200,No log,0.682733
225,No log,0.676643
250,No log,0.673809


***** Running Evaluation *****
  Num examples = 1390
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1390
  Batch size = 16
Saving model checkpoint to assessed_model/checkpoint-50
Configuration saved in assessed_model/checkpoint-50/config.json
Model weights saved in assessed_model/checkpoint-50/pytorch_model.bin
tokenizer config file saved in assessed_model/checkpoint-50/tokenizer_config.json
Special tokens file saved in assessed_model/checkpoint-50/special_tokens_map.json
Copy vocab file to assessed_model/checkpoint-50/spiece.model
***** Running Evaluation *****
  Num examples = 1390
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1390
  Batch size = 16
Saving model checkpoint to assessed_model/checkpoint-100
Configuration saved in assessed_model/checkpoint-100/config.json
Model weights saved in assessed_model/checkpoint-100/pytorch_model.bin
tokenizer config file saved in assessed_model/checkpoint-100/tokenizer_config.json
Special tokens file saved

TrainOutput(global_step=258, training_loss=1.3486239411110101, metrics={'train_runtime': 291.2278, 'train_samples_per_second': 57.254, 'train_steps_per_second': 0.886, 'total_flos': 1423248769843200.0, 'train_loss': 1.3486239411110101, 'epoch': 2.99})

In [16]:
tokenizer.decode(model.generate(tokenizer(["Если кто не знает -- ray marching это техника рендеринга сцен, в которой лучи итеративно \"прощупывают\" сцену с помощью лишь функции расстояния до ближайшего объекта. Если алгоритм рассматривает луч в какой-то точке и знает расстояние до сцены, то это значит, что можно пройти это расстояние в направлении луча и ничего не задеть. Алгоритм проходит ровно это расстояние, и приступает к следующей итерации."], return_tensors="pt").input_ids.cuda())[0].cpu(), skip_special_tokens=True)

'если кто не знает, что за ебала техника рендеринга сцен, в которой алгоритм'

In [18]:
model.save_pretrained("assessed_release")

Configuration saved in assessed_release/config.json
Model weights saved in assessed_release/pytorch_model.bin


In [18]:
test_input = "на его место встал логичный Zeliboba Score"

tokenizer.decode(model.generate(tokenizer([test_input], return_tensors="pt").input_ids.cuda(),
                                do_sample=True,
                                top_k=10,
                                temperature=1.5)[0].cpu(), skip_special_tokens=True)

'на на его место встал логичный пиздеец, и с его места встал логичный'

In [4]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruT5-base")

model = AutoModelForSeq2SeqLM.from_pretrained("assessed_release").cuda()

df = pd.read_csv("detoxified.csv")